# **미니프로젝트 4차 1대1 문의 내용 유형 분류기**
# 단계1 : 데이터 탐색

### 문제 정의
> 1:1 문의 내용 분류 문제<br>
> 1. 문의 내용 분석
> 2. 문의 내용 분류 모델 성능 평가
### 학습 데이터
> * 1:1 문의 내용 데이터 : train.csv

### 변수 소개
> * text : 문의 내용
> * label : 문의 유형

### References
> * 한국어 처리
>> * [konlpy - 한국어 처리 라이브러리](https://konlpy.org/ko/latest/)
>> * [한국어 품사 태그 비교표](https://docs.google.com/spreadsheets/d/1OGAjUvalBuX-oZvZ_-9tEfYD2gQe7hTGsgUpiiBSXI8/edit#gid=0)
>> * [한국어 품사 태깅 성능 비교](https://konlpy.org/ko/latest/morph/#comparison-between-pos-tagging-classes)
>> * [한국어 시스템 사전](https://konlpy.org/ko/latest/data/#corpora)

> * 자연어 처리
>> * [NLTK](https://www.nltk.org/book/)
>> * [gensim](https://radimrehurek.com/gensim/)
>> * [Google guide](https://developers.google.com/machine-learning/guides/text-classification/step-2)
>> * [WordCloud](https://amueller.github.io/word_cloud/)

## 1. 개발 환경 설정

* 세부 요구사항
  - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
  - 필요하다고 판단되는 라이브러리를 추가하세요.
  - konlpy, mecab 설치 후 형태소 분석 함수 생성
  - mecab 설치할 때 윈도우 pc에서 설치는 다른 방법으로 진행
  - 윈도우 환경일 경우 KoNLPy의 라이브러리 설치가 제대로 이루어지지 않을 수 있습니다
  - 윈도우 설치를 위한 참고 링크
    - https://liveyourit.tistory.com/56

### 1-1. 라이브러리 설치

In [1]:
# 필요 라이브러리부터 설치할께요.
!pip install konlpy pandas seaborn gensim wordcloud python-mecab-ko wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.6/575.6 KB 32.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 15.4 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=aa18586a21d956413c87c8b48169962e96936d3d02e940f7fcc6c8e5328a8578
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


### 1-2. 라이브러리 import

In [24]:
from mecab import MeCab
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from wordcloud import WordCloud
from collections import Counter
import wget, os
from konlpy.utils import pprint
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *

### 1-4. 구글드라이브 연결(Colab)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2.데이터 불러오기

* 주어진 데이터
 - 학습 및 검증용 데이터 : train.csv

### 2-1. 데이터 로딩

* 다음 데이터를 불러옵니다.
    * 학습 및 검증용 데이터 : train.csv
    * shape를 확인합니다.

In [4]:
data_path = '/content/drive/MyDrive/에이블스쿨/실습파일/2023.04.03_미니프로젝트4차_실습자료/data/train.csv'
data = pd.read_csv(data_path)

In [5]:
data.shape

(3706, 2)

# 전처리

In [11]:
label_dict = {
    '코드1': 0,
    '코드2': 0,
    '웹': 1,
    '이론': 2,
    '시스템 운영': 3,
    '원격': 4
}

data = data.replace(label_dict)
data.head()

,text,label
0,"self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, ...",0
1,현재 이미지를 여러개 업로드 하기 위해 자바스크립트로 동적으로 폼 여러개 생성하는데...,1
2,glob.glob(PATH) 를 사용할 때 질문입니다.\n\nPATH에 [ ] 가 ...,0
3,"tmpp = tmp.groupby(by = 'Addr1', as_index=Fals...",0
4,filename = TEST_IMAGE + str(round(frame_sec)) ...,0


In [10]:
mecab = MeCab()
pprint(mecab.pos("하하 이거 어떻게 성능을 올리지?"))

[('하하', 'MAG'),
 ('이거', 'NP'),
 ('어떻게', 'MAG'),
 ('성능', 'NNG'),
 ('을', 'JKO'),
 ('올리', 'VV'),
 ('지', 'EF'),
 ('?', 'SF')]


In [12]:
result = []
for i in data['text']:
    temp = []
    for morph, tag in mecab.pos(i):
        temp.append(f'{morph}/{tag}')
    result.append(' '.join(temp))
result[:5]

['self/SL ./SY convs/SL 1/SN =/SY nn/SL ./SY ModuleList/SL (/SSO [/SSO nn/SL ./SY Conv/SL 2/SN d/SL (/SSO 1/SN ,/SC Co/SL ,/SC (/SSO K/SL ,/SC 100/SN )/SSC )/SSC for/SL K/SL in/SL Ks/SL ]/SSC )/SSC 1/SN ./SF 여기/NP 서/JKB 커널/NNG 사이즈/NNG K/SL ,/SC 100/SN 은/NNG 한/MM 단어/NNG 의/JKG 최대/NNG 길이/NNG 가/JKS 100/SN 이/JKS 넘/VV 지/EC 않/VX 는다는/ETM 가정/NNG 으로/JKB 그냥/MAG 100/SN 으로/JKB 선언/NNG 한/XSA+ETM 것/NNB 인지/VCP+EC 궁금/XR 합니다/XSA+EF ./SF',
 '현재/MAG 이미지/NNG 를/JKO 여러/MM 개/NNBC 업로드/NNG 하/VV 기/ETN 위해/VV+EC 자바/NNG 스크립트/NNG 로/JKB 동적/NNG 으로/JKB 폼/NNG 여러/MM 개/NNBC 생성/NNG 하/XSV 는/ETM 데/NNB 클릭/NNG 해도/XSV+EC 생/XR 성/XSN 이/JKS 안/MAG 되/VV 네요/EF ./SF ../SY 막혀서/VV+EC 뭘/NP+JKO 할/VV+ETM 수/NNB 가/JKS 없/VA 네요/EF ./SF ./SY 혹시나/MAG 해서/VV+EC 기본/NNG 예제/NNG 코드/NNG 로/JKB 만/JX 간단히/MAG 돌려도/VV+EC 안/MAG 됩니다/VV+EF ./SF 이유/NNG 가/JKS 뭘까요/NP+VCP+EF',
 'glob/SL ./SY glob/SL (/SSO PATH/SL )/SSC 를/JKO 사용/NNG 할/XSV+ETM 때/NNG 질문/NNG 입니다/VCP+EF ./SF PATH/SL 에/JKB [/SSO ]/SSC 가/XPN 포함/NNG 되/XSV 면/EC 제대로/MAG 작동/NNG 하/XSV 지/EC 않/VX 는/ETM 것/NNB 같/VA

In [13]:
data['morphs/tag'] = result

In [16]:
x_train, x_test, y_train, y_test = train_test_split(
    data['morphs/tag'], data['label'], test_size=0.2, 
    random_state=2023, stratify=data['label'])

In [19]:
Tfidf_vect = TfidfVectorizer()

In [20]:
Tfidf_vect.fit(x_train)

TfidfVectorizer()

In [21]:
x_train_tfidf = Tfidf_vect.transform(x_train)
x_test_tfidf = Tfidf_vect.transform(x_test)

In [29]:
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(x_train_tfidf, y_train)

LogisticRegression(C=20, max_iter=1000)

In [30]:
y_pred = lr.predict(x_test_tfidf)
accuracy_score(y_test, y_pred)

0.839622641509434

---

In [26]:
result = []
for i in data['text']:
    temp = []
    for morph, tag in mecab.pos(i):
        temp.append(morph)
    result.append(' '.join(temp))
result[:5]

['self . convs 1 = nn . ModuleList ( [ nn . Conv 2 d ( 1 , Co , ( K , 100 ) ) for K in Ks ] ) 1 . 여기 서 커널 사이즈 K , 100 은 한 단어 의 최대 길이 가 100 이 넘 지 않 는다는 가정 으로 그냥 100 으로 선언 한 것 인지 궁금 합니다 .',
 '현재 이미지 를 여러 개 업로드 하 기 위해 자바 스크립트 로 동적 으로 폼 여러 개 생성 하 는 데 클릭 해도 생 성 이 안 되 네요 . .. 막혀서 뭘 할 수 가 없 네요 . . 혹시나 해서 기본 예제 코드 로 만 간단히 돌려도 안 됩니다 . 이유 가 뭘까요',
 'glob . glob ( PATH ) 를 사용 할 때 질문 입니다 . PATH 에 [ ] 가 포함 되 면 제대로 작동 하 지 않 는 것 같 아 질문 드립니다 . 제공 해 주 신 파일 에 AIVLE _ 3 rd [ Team ] 폴더 가 포함 되 어 있 는데 , 이 폴더 안 에 는 glob 이 정상 적 으로 작동 하 지 않 습니다 .',
 "tmpp = tmp . groupby ( by = ' Addr 1 ', as _ index = False ) [ [ ' Category ']]. sum ( ) plt . figure ( figsize =( 15 , 5 ) ) plt . bar ( tmpp [ ' Addr 1 '], tmpp [ ' Category ']/ 1000 ) plt . ticklabel _ format ( axis =' y ', style =' plain ') plt . show ( ) 오류 : unsupported operand type ( s ) for / : ' str ' and ' int ' 어떻 게 해결 하 면 좋 을까요 ? !!!!",
 'filename = TEST _ IMAGE + str ( round ( frame _ sec ) ) + ". jpg " 이렇게 하 면 test _ path 안 에 image 폴더 안 에 이미지 들 이 저장 되

In [31]:
data['morphs'] = result

In [33]:
x_train, x_test, y_train, y_test = train_test_split(
    data['morphs'], data['label'], test_size=0.2, 
    random_state=2023, stratify=data['label'])

In [36]:
Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(x_train)
x_train_tfidf = Tfidf_vect.transform(x_train)
x_test_tfidf = Tfidf_vect.transform(x_test)

In [37]:
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(x_train_tfidf, y_train)
y_pred = lr.predict(x_test_tfidf)
accuracy_score(y_test, y_pred)

0.8423180592991913

---

In [44]:
import re

def remove_i(text):
    result = re.sub(r"[^\uAC00-\uD7A3a-zA-Z\s]", "", text)
    return result

data['removed_text'] = data['text'].apply(remove_i)

In [45]:
result = []
sl_count = []
for i in data['removed_text']:
    temp = []
    count = 0
    for morph, tag in mecab.pos(i):
        if tag == 'SL':
            count+=1
        else:
            temp.append(morph)
    result.append(' '.join(temp))
    sl_count.append(count)
result[:5], sl_count[:5]

(['여기 서 커널 사이즈 은 한 단어 의 최대 길이 가 이 넘 지 않 는다는 가정 으로 그냥 으로 선언 한 것 인지 궁금 합니다',
  '현재 이미지 를 여러 개 업로드 하 기 위해 자바 스크립트 로 동적 으로 폼 여러 개 생성 하 는 데 클릭 해도 생 성 이 안 되 네요 막혀서 뭘 할 수 가 없 네요 혹시나 해서 기본 예제 코드 로 만 간단히 돌려도 안 됩니다 이유 가 뭘까요',
  '를 사용 할 때 질문 입니다 에 가 포함 되 면 제대로 작동 하 지 않 는 것 같 아 질문 드립니다 제공 해 주 신 파일 에 폴더 가 포함 되 어 있 는데 이 폴더 안 에 는 이 정상 적 으로 작동 하 지 않 습니다',
  '오류 어떻게 해결 하 면 좋 을까요',
  '이렇게 하 면 안 에 폴더 안 에 이미지 들 이 저장 되 는 걸로 이해 했 는데 이렇게 실행 하 니 폴더 에 이미지 가 저장 되 어서 질문 드립니다'],
 [9, 0, 4, 17, 7])

In [47]:
data['rm_morph'] = result
data['sl_count'] = sl_count

In [48]:
data['sl_count'].max(), data['sl_count'].min()

(1950, 0)

In [49]:
data['sl_count'] = data['sl_count'] / data['sl_count'].max()

In [50]:
x = data.loc[:, ['rm_morph', 'sl_count']]
y = data['label']

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, 
    random_state=2023, stratify=y)

In [54]:
Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(x_train['rm_morph'])
x_train_tfidf = Tfidf_vect.transform(x_train['rm_morph'])
x_test_tfidf = Tfidf_vect.transform(x_test['rm_morph'])

In [55]:
x_train_tfidf.shape

(2964, 4295)

In [56]:
x_train['sl_count'].shape

(2964,)

In [75]:
a = x_train_tfidf.toarray()
b = x_train['sl_count'].values.reshape(-1, 1)

x_train = np.hstack((a, b))

a = x_test_tfidf.toarray()
b = x_test['sl_count'].values.reshape(-1, 1)

x_test = np.hstack((a, b))

In [76]:
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
accuracy_score(y_test, y_pred)

0.7506738544474394